In [5]:
import numpy as np 
import pandas as pd 
#import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import wilcoxon
from scipy.stats import binomtest
from statsmodels.stats.multitest import multipletests #para corrección bonferroni
from scipy.stats import ttest_ind #prueba t para dos muestras independientes 
from itertools import  combinations #combinaciones para prueba t de comparaciones multiples
from scipy.stats import ttest_rel #Prueba t para muestras pareadas o no independientes 
from scipy.stats import mannwhitneyu #Prueba U de Mann-Whitney 

def IQR(tabla,variable):
    '''Calcula el rango intercuartil (IRQ) de una serie de datos de un dataframe
    variable: str, nombre de la columna o fila del data frame que se quiere calcular el IQR'''
    try:
        variable = str(variable)
    except TypeError:
        raise TypeError('El argumento debe ser un string')
    Q1=tabla[variable].quantile(0.25, axis=1)
    Q2=tabla[variable].quantile(0.5, axis=1)
    Q3=tabla[variable].quantile(0.75, axis=1)
    return Q3-Q1

In [12]:
#Cargar el archivo al computador y crear un dataframe 
data=pd.read_csv('FLIR_groups1and2.csv', sep=';')
# Seprar los datos en tablas para su posterior análisis
tabla1 = data.iloc[:, 2:30].copy()
tabla2 = data.iloc[:, 30:59].copy()
tabla3 = data.iloc[:, 58:86].copy()
tabla4 = data.iloc[:, 114:125].copy()
tabla0 = data.iloc[:, 0:1].copy()
tablaMAX = data.loc[:,['Unnamed: 3','Unnamed: 31','Unnamed: 59','Unnamed: 87']]
# print(tabla0.keys())
# print(tabla1.keys())
# print(tabla2.keys())
# print(tabla3.keys())
# print(tabla4.keys())
tablaMAX = tablaMAX.drop([0, 1])
tablaMAX = tablaMAX.apply(pd.to_numeric, errors='coerce')
tablaMAX.fillna(tablaMAX, inplace=True)
# Cambiamos los indices de la primera columna 
tablaMAX[''] = range(1, len(tablaMAX) + 1)  # Add a new column with the range
tablaMAX.set_index('', inplace=True)       # Set the new column as the index

TablaAnalisis = pd.concat([tabla0,tabla4], axis=1)
TablaAnalisis = TablaAnalisis.rename(columns={
    'Unnamed: 0': 'SubjectID', 
    'Unnamed: 115': 'aveOralM',
    'Unnamed: 116': 'Gender',
    'Unnamed: 117': 'Age',
    'Unnamed: 118': 'Ethnicity',
    'Unnamed: 119': 'T_atm',
    'Unnamed: 120': 'Humidity',
    'Unnamed: 122': 'Cosmetics'  # Fixed syntax - removed extra 'Unnamed 123'
})
TablaAnalisis.pop('Other parameters: ')
#Eliminamos las dos primeras filas que no contienen datos relevantes
TablaAnalisis = TablaAnalisis.drop([0, 1])
TablaAnalisis[''] = range(1, len(tablaMAX) + 1)  # Add a new column with the range
TablaAnalisis.set_index('', inplace=True)  
TablaAnalisis["ProMax"] = tablaMAX.mean(axis=1)

print(TablaAnalisis.keys())



Index(['SubjectID', 'aveOralM', 'Gender', 'Age', 'Ethnicity', 'T_atm',
       'Humidity', 'Unnamed: 121', 'Cosmetics', 'Unnamed: 123', 'Unnamed: 124',
       'ProMax'],
      dtype='object')


In [13]:
#Edad 
# Convertir valores de 'Gender' a binarios: Male -> 1, Female -> 0
TablaAnalisis.loc['Gender']= TablaAnalisis['Gender'].replace({'Male': 1, 'Female': 0})
# Contar la cantidad de valores binarios para 'Gender' donde Gender == 1 (Male)
df = sum(TablaAnalisis['Gender'] == 1)


C:\Users\maria\AppData\Local\Temp\ipykernel_18032\973179837.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  TablaAnalisis.loc['Gender']= TablaAnalisis['Gender'].replace({'Male': 1, 'Female': 0})


In [ ]:
# TablaAnalisis['aveOralM'] = TablaAnalisis['aveOralM'].str.extract(r'(\d+\.\d+)').astype(float)
# # Si hay valores nulos, podrías querer rellenarlos (aunque no es obligatorio para calcular la media)

# # Ahora calculas la media
# Media = TablaAnalisis['aveOralM'].mean()
TablaAnalisisT=TablaAnalisis.T
print(TablaAnalisisT.keys())



# TablaAnalisisT['Media']=TablaAnalisisT.fillna(value=TablaAnalisisT['aveOralM'].mean(), inplace=True)
# TablaAnalisisT[]= TablaAnalisisT.median(axis=1)
# TablaAnalisisT['Moda']= TablaAnalisisT.mode(axis=1)[0]
# TablaAnalisisT['Desviacion']= TablaAnalisisT.std(axis=1) 
# TablaAnalisisT['MAD']='¿Respecto a que?' 
# TablaAnalisisT['Varianza']= TablaAnalisisT.var(axis=1)


LimiteInferior=TablaAnalisisT['Media']-1.5*TablaAnalisisT['Desviacion']
LimiteSuperior=TablaAnalisisT['Media']+1.5*TablaAnalisisT['Desviacion']
'''Media de aveOralM'''
# Si 'aveOralM' contiene strings como 'valor °C' o 'valor texto', primero extraemos el número decimal

